# mini-batch梯度下降

我们在之前实战编程中一直都是用梯度下降算法来更新参数来使成本函数最小化。

在梯度下降的每一步中，都会朝着某个方向来更新参数以找到成本函数的最小值处。

首先加载一些系统工具库以及一些我们自定义的库。

In [18]:
import numpy as np
from testCases import *

%matplotlib inline

## 1. 梯度下降

之前我们一直使用的梯度下降算法只是一个简单的基础优化算法。

如果每次梯度下降的学习对象都是所有的样本，那么这个梯度下降算法就叫做Batch梯度下降。

下面这个就是梯度下降中用来更新参数的函数。

In [19]:
def update_parameters_with_gd(parameters, grads, learning_rate):
    # 获取神经网络的层数。这里除以2是因为字典里面包含了w和b两种参数。
    L = len(parameters) // 2

    # 遍历每一层
    for l in range(L):
        # 下面使用l + 1，是因为l是从0开始的，而我们的参数字典是从1开始的
        parameters["W" + str(l + 1)] = parameters["W" + str(l + 1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] = parameters["b" + str(l + 1)] - learning_rate * grads["db" + str(l + 1)]
        
    return parameters

In [20]:
parameters, grads, learning_rate = update_parameters_with_gd_test_case()

parameters = update_parameters_with_gd(parameters, grads, learning_rate)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 1.63535156 -0.62320365 -0.53718766]
 [-1.07799357  0.85639907 -2.29470142]]
b1 = [[ 1.74604067]
 [-0.75184921]]
W2 = [[ 0.32171798 -0.25467393  1.46902454]
 [-2.05617317 -0.31554548 -0.3756023 ]
 [ 1.1404819  -1.09976462 -0.1612551 ]]
b2 = [[-0.88020257]
 [ 0.02561572]
 [ 0.57539477]]


与batch梯度下降相对的另一个极端是随机梯度下降。

当mini-batch梯度下降的子数据集中只有1个样本时，就是随机梯度下降。

这3种梯度下降算法的参数更新算法都是一样的（都是用上面我们实现的更新函数）。

不同的是，随机梯度下降每次的学习对象只有一个样本，而batch梯度下降每次的学习对象是所有样本。

下面的两个代码段展示了batch梯度下降和随机梯度下降的差别。

**batch梯度下降**：

``` python
X = data_input
Y = labels
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):
    a, caches = forward_propagation(X, parameters)
    cost = compute_cost(a, Y)
    grads = backward_propagation(a, caches, parameters)
    parameters = update_parameters(parameters, grads)
        
```

**随机梯度下降**：

```python
X = data_input
Y = labels
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):
    # 遍历循环每一个样本
    for j in range(0, m):
        a, caches = forward_propagation(X[:,j], parameters)
        cost = compute_cost(a, Y[:,j])
        grads = backward_propagation(a, caches, parameters)
        parameters = update_parameters(parameters, grads)
```

在随机梯度下降中，每次都只使用一个样本来进行梯度下降。如果你的数据集非常大，那么使用随机梯度下降可能会比batch梯度下降更快。但是随机梯度下降的方向会很不稳定。如下图所示，左图是随机梯度下降，右图是batch梯度下降。

<img src="images/kiank_sgd.png" style="width:750px;height:250px;">
<caption><center>图1：随机梯度下降 vs batch梯度下降</center></caption>

另外有些同学应该也注意到了，随机梯度下降要使用3个循环：

1. 遍历每一次梯度下降iteration。
2. 遍历所有样本。
3. 遍历神经网络的每一层。

在实际编程中，往往使用mini-batch梯度下降会比batch梯度下降和随机梯度下降都要高效，使神经网络学习得更快。下面两个图展示了随机梯度下降和mini-batch梯度下降的学习路径。

<img src="images/kiank_minibatch.png" style="width:750px;height:250px;">
<caption><center>图2：随机梯度下降 vs mini-batch梯度下降</center></caption>

**下面三点大家需要牢记**：
- 这3个梯度下降的区别仅仅在于它们每次学习的样本数量不同。
- 无论是哪种梯度下降，学习率都是必须要精心调的。
- 通常来说，如果数据集很大，那么mini-batch梯度下降会比另外2种要高效。

## 2. mini-batch梯度下降

下面我们来学习如何从训练集(X, Y)中创建mini-batch，分两个步骤：

- **洗牌**：如下图所示，将训练集(X, Y)进行洗牌——将样本随机调换位置。这样一来，每一次的子训练集中都包含着不同的样本。下图中每一列就代表了一个训练样本。注意，在洗牌时，X和Y是被绑在一起进行洗牌的，也就是说，在洗牌后，之前第$i^{th}$列中的X还是与之前$i^{th}$列的Y是一对。

    <img src="images/kiank_shuffle.png" style="width:550px;height:300px;">

- **分割**：将洗牌后的训练集划分为一个个小的子训练集。这里我们每个子训练集中有64个样本，也就是说mini_batch_size是64。注意，有时候训练集无法被mini_batch_size整除，那么最后一个子训练集里面的样本数就会小于mini_batch_size，这个是没有关系的。 

    <img src="images/kiank_partition.png" style="width:550px;height:300px;">

下面的函数实现了上面两步。

In [21]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    np.random.seed(seed)            
    m = X.shape[1]  # 获取样本数量
    mini_batches = []
        
    # 第一步：洗牌训练集
    # 对0~(m-1)的序列进行随机排序
    # 如果m是3，那么结果可能为[2, 0, 1]
    permutation = list(np.random.permutation(m))
    # X[行,列]
    # X[:,n]：表示把所有行中的第n个元素取出
    # X[n,:]：表示把第n行的所有元素取出
    # X[:,n:m]：表示把所有行中的从n到m-1的元素取出
    # X[n:m,:]：表示把从n到m-1行的所有元素取出
    # X[:,[3,2,1]]：表示把所有行的第3、2、1个元素按顺序取出，或者说是把第3、2、1列的元素取出
    shuffled_X = X[:, permutation] # 将X按permutation列表里的索引顺序进行重排列
    # 同时要保证标签和数据在洗牌前后还是一对
    # reshape((1,m))用于保证维度正确，因为有时候Python内部会自动改变维度
    shuffled_Y = Y[:, permutation].reshape((1,m))

    # 第二步：分割洗牌后的训练集
    # 获取子训练集的个数（不包括后面不满mini_batch_size的那个子训练集）
    num_complete_minibatches = math.floor(m/mini_batch_size)
    # 循环取出满mini_batch_size的子训练集
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:,k * mini_batch_size:(k + 1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:,k * mini_batch_size:(k + 1) * mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # 取出后面不满mini_batch_size的那个子训练集
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:,num_complete_minibatches * mini_batch_size:]
        mini_batch_Y = shuffled_Y[:,num_complete_minibatches * mini_batch_size:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [22]:
X_assess, Y_assess, mini_batch_size = random_mini_batches_test_case()
mini_batches = random_mini_batches(X_assess, Y_assess, mini_batch_size)

print("第一个mini_batch_X的维度: " + str(mini_batches[0][0].shape))
print("第二个mini_batch_X的维度: " + str(mini_batches[1][0].shape))
print("第三个mini_batch_X的维度: " + str(mini_batches[2][0].shape))
print("第一个mini_batch_Y的维度: " + str(mini_batches[0][1].shape))
print("第二个mini_batch_Y的维度: " + str(mini_batches[1][1].shape)) 
print("第三个mini_batch_Y的维度: " + str(mini_batches[2][1].shape))

第一个mini_batch_X的维度: (12288, 64)
第二个mini_batch_X的维度: (12288, 64)
第三个mini_batch_X的维度: (12288, 20)
第一个mini_batch_Y的维度: (1, 64)
第二个mini_batch_Y的维度: (1, 64)
第三个mini_batch_Y的维度: (1, 20)


**大家需要记住下面几点**：
- 洗牌和分割是实现mini-batch梯度下降的两个重要步骤。
- mini-batch的大小一般选择2的次方。